In [6]:

import pandas as pd
import numpy as np
from datetime import date
import urllib.request
import urllib3.request

import json
import datetime
from datetime import datetime as dt
from datetime import timedelta
import warnings
import requests
from pytz import timezone
import threading
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import CancelledError
pd.pandas.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
import plotly.graph_objs as go
import plotly.offline as pyo

In [10]:
#Crypto symbol fetcher of the 20 coins
def crypto_sym_generator(end_date):
    url="https://web-api.coinmarketcap.com/v1/cryptocurrency/listings/historical?convert=USD&date="+str(end_date)+"&limit=20"
    m=requests.get(url)
    n=json.loads(m.text)
    crypto_sym_list=[]
    for i in range(0,20):
        r=n['data'][i]['symbol']
        crypto_sym_list.append(r)
    return crypto_sym_list
#     crypto_sym_list=[n['data'][i]['symbol'] for i in n['data']]
crypto_sym_list=crypto_sym_generator("2023-03-25")
crypto_sym_list

['BTC',
 'ETH',
 'USDT',
 'BNB',
 'USDC',
 'XRP',
 'ADA',
 'DOGE',
 'MATIC',
 'BUSD',
 'SOL',
 'DOT',
 'LTC',
 'SHIB',
 'TRX',
 'AVAX',
 'DAI',
 'WBTC',
 'LINK',
 'LEO']

In [11]:
# fetchinf the Close data of cypto assets
def data_fetcher(symbol):
    url="https://min-api.cryptocompare.com/data/v2/histoday?fsym="+symbol+"&tsym=USD&limit=1095"
    m=requests.get(url)
    n=json.loads(m.text)
    df=pd.DataFrame(n['Data']['Data'])
    df['conversionSymbol']=symbol
    df=df.drop(['high','low','open','volumefrom','volumeto','conversionType','conversionSymbol'],axis=1)
    df['time']=pd.to_datetime(df['time'],unit='s')
    df=df.add_suffix("_"+symbol)
    df=df.rename(columns={"time_"+symbol:'time'})
#     df=df.drop(['high','low','open','volumefrom','volumeto','conversionType','conversionSymbol'],axis=1)
    return df
# data_fetcher('ETH')

In [13]:
crypto_sym_list=['BTC']

In [14]:
#using thread to download the data
def thread_initiator():
    result=pd.DataFrame()
    with ThreadPoolExecutor(10) as executor: 
            try: 
                a=executor.map(data_fetcher, crypto_sym_list)
                print(a)
                for r in a:
                    result=pd.concat([result,r.set_index('time')],axis=1)
                return result                
            except CancelledError:
                print('No exception, task was cancelled')

a=thread_initiator()
df1=a.copy()

<generator object Executor.map.<locals>.result_iterator at 0x0000020BFAB93A50>


In [15]:
df1_long_returns=df1.pct_change()
df1_long_returns=df1_long_returns.replace([np.inf,-np.inf],0)
df1_short_returns=-1 *df1_long_returns
df1_long_returns

,close_BTC
time,
2020-04-03,NaN
2020-04-04,0.019614
2020-04-05,-0.013716
2020-04-06,0.083399
2020-04-07,-0.019824
...,...
2023-03-30,-0.011229
2023-03-31,0.015687
2023-04-01,-0.000421


In [11]:
df_3D_mean = df1.rolling(window=3).mean()
df_30D_mean = df1.rolling(window=30).mean()

In [12]:
df_avg=df_3D_mean/df_30D_mean
df_monthly_avg=df_avg.resample('M').last()
df_monthly_avg

In [13]:
# ade=df_monthly_avg.iloc[1].sort_values(ascending=False).dropna()
# ade

close_SOL      1.461470
close_ADA      1.313836
close_MATIC    1.205405
close_ETH      1.202415
close_WBTC     1.201413
close_TRX      1.177004
close_BTC      1.161970
close_DOGE     1.157413
close_LINK     1.150511
close_XRP      1.135459
close_BNB      1.113598
close_LTC      1.093952
close_LEO      1.023960
close_BUSD     1.000073
close_USDC     0.999927
close_USDT     0.999837
close_DAI      0.996727
Name: 2020-04-30 00:00:00, dtype: float64

In [14]:
def long_short_ranker():
    df=pd.DataFrame(columns=['long_list','short_list'])
    df['time']=df_monthly_avg.index
    df.set_index(['time'],inplace=True)
    for i in range(0,len(df_monthly_avg+1)):
        df['long_list'].iloc[i]=list(df_monthly_avg.iloc[i].sort_values(ascending=False).dropna().index[:5])
        df['short_list'].iloc[i]=list(df_monthly_avg.iloc[i].sort_values(ascending=False).dropna().index[-5:])
    return df
a=long_short_ranker()
a

,long_list,short_list
time,,
2020-03-31,[],[]
2020-04-30,"[close_SOL, close_ADA, close_MATIC, close_ETH,...","[close_LEO, close_BUSD, close_USDC, close_USDT..."
2020-05-31,"[close_ADA, close_MATIC, close_ETH, close_LEO,...","[close_USDC, close_BUSD, close_DAI, close_XRP,..."
2020-06-30,"[close_SOL, close_LINK, close_LEO, close_ADA, ...","[close_MATIC, close_LTC, close_BNB, close_XRP,..."
2020-07-31,"[close_SOL, close_ETH, close_LTC, close_XRP, c...","[close_DOGE, close_USDT, close_BUSD, close_USD..."
2020-08-31,"[close_DOT, close_SOL, close_TRX, close_LINK, ...","[close_BTC, close_WBTC, close_DOGE, close_XRP,..."
2020-09-30,"[close_AVAX, close_BNB, close_ADA, close_LEO, ...","[close_LTC, close_SOL, close_DOT, close_LINK, ..."
2020-10-31,"[close_BTC, close_WBTC, close_LTC, close_LINK,...","[close_XRP, close_AVAX, close_ADA, close_MATIC..."
2020-11-30,"[close_XRP, close_ADA, close_ETH, close_DOGE, ...","[close_USDC, close_BUSD, close_USDT, close_DAI..."


In [ ]:
# def positions_long_short_returns():
#     df=pd.DataFrame(columns=['long_position_returns','short_position_returns'])
#     df['time']=df1_long_returns.index
#     df.set_index(['time'],inplace=True)
#     print(df)
# #     for i in range(0,len(df-2)):
# #         print(i)
# #         df['long_position_returns'].iloc[i+1]=((df1_long_returns.loc[a.index[i]:a.index[i+1],a['long_list'].iloc[i]].sum(axis=1))+1).cumprod()[-1]
# #         df['short_position_returns'].iloc[i+1]=((df1_long_returns.loc[a.index[i]:a.index[i+1],a['short_list'].iloc[i]].sum(axis=1))+1).cumprod()[-1]
# #     return df
# red=positions_long_short_returns()
# red

In [ ]:
# df=pd.DataFrame(columns=['long_position_returns','short_position_returns'])
# df['time']=df1_long_returns.index
# df.set_index(['time'],inplace=True)
# df

In [24]:
# df1_long_returns.loc[a.index[13]:a.index[14],a['long_list'].iloc[13]].sum(axis=1)
df1_long_returns.loc[a.index[13]:a.index[14],a['long_list'].iloc[13]]

,close_MATIC,close_DOGE,close_SOL,close_BNB,close_ETH
time,,,,,
2021-04-30,-0.078401,0.109145,0.007521,0.037559,0.006344
2021-05-01,-0.042535,0.156324,0.156520,-0.004092,0.061903
2021-05-02,0.048243,-0.044723,-0.053852,0.002626,0.001418
2021-05-03,-0.034001,0.180578,-0.018120,0.087307,0.162410
2021-05-04,-0.114016,0.220712,-0.074251,-0.098194,-0.055464
2021-05-05,0.149830,0.214591,0.054878,0.068750,0.088788
2021-05-06,-0.058366,-0.111722,-0.032681,-0.027172,-0.010519
2021-05-07,-0.013498,0.186167,-0.006895,-0.015132,-0.002228
2021-05-08,0.129251,-0.076298,0.058320,0.035835,0.124080


In [ ]:
# # abs=pd.DataFrame(columns=['time','time_series_rebal'])
# rest=pd.DataFrame()

# for i in range(1,36):
#     abs=pd.DataFrame(df1_long_returns.loc[a.index[i]:a.index[i+1],a['long_list'].iloc[i]].sum(axis=1),columns=['time_series_rebal'])
#     rest=pd.concat([rest,abs])
# #     rest.append(abs)
# print(pd.DataFrame(rest))


In [30]:
def time_series_long():
    rest=pd.DataFrame()

    for i in range(0,36):
        abs=pd.DataFrame(df1_long_returns.loc[a.index[i]+timedelta(days=1):a.index[i+1],a['long_list'].iloc[i]].sum(axis=1),columns=['time_series_rebal_long'])
        rest=pd.concat([rest,abs])
    rest['cum_time_series_rebal_long']=1+rest['time_series_rebal_long']
    rest['cum_ts_rebal_long']=rest['cum_time_series_rebal_long'].cumprod()
    return rest
ts_long=time_series_long()
# ts_long.to_csv("ts_long.csv")
ts_long

,time_series_rebal_long,cum_time_series_rebal_long,cum_ts_rebal_long
time,,,
2020-04-01,0.000000,1.000000,1.000000
2020-04-02,0.000000,1.000000,1.000000
2020-04-03,0.000000,1.000000,1.000000
2020-04-04,0.000000,1.000000,1.000000
2020-04-05,0.000000,1.000000,1.000000
...,...,...,...
2023-03-22,-0.236654,0.763346,-0.001371
2023-03-23,0.271494,1.271494,-0.001743
2023-03-24,-0.178783,0.821217,-0.001431


In [31]:
def time_series_short():
    rest=pd.DataFrame()

    for i in range(0,36):
        abs=pd.DataFrame(df1_short_returns.loc[a.index[i]+timedelta(days=1):a.index[i+1],a['short_list'].iloc[i]].sum(axis=1),columns=['time_series_rebal_short'])
        rest=pd.concat([rest,abs])
    rest['cum_time_series_rebal_short']=1+rest['time_series_rebal_short']
    rest['cum_ts_rebal_short']=rest['cum_time_series_rebal_short'].cumprod()
    return rest
ts_short=time_series_short()
# ts_short.to_csv("ts_short.csv")
ts_short

,time_series_rebal_short,cum_time_series_rebal_short,cum_ts_rebal_short
time,,,
2020-04-01,0.000000,1.000000,1.000000e+00
2020-04-02,0.000000,1.000000,1.000000e+00
2020-04-03,0.000000,1.000000,1.000000e+00
2020-04-04,0.000000,1.000000,1.000000e+00
2020-04-05,0.000000,1.000000,1.000000e+00
...,...,...,...
2023-03-22,0.178114,1.178114,5.189995e-18
2023-03-23,-0.178330,0.821670,4.264463e-18
2023-03-24,0.163062,1.163062,4.959836e-18


In [33]:
final_df=pd.concat([ts_long['time_series_rebal_long'],ts_short['time_series_rebal_short']],axis=1)
final_df['total']=final_df.sum(axis=1)
final_df['return_total']=(1+final_df['total'])
final_df['cum_return_total']=final_df['return_total'].cumprod()
final_df

,time_series_rebal_long,time_series_rebal_short,total,return_total,cum_return_total
time,,,,,
2020-04-01,0.000000,0.000000,0.000000,1.000000,1.000000e+00
2020-04-02,0.000000,0.000000,0.000000,1.000000,1.000000e+00
2020-04-03,0.000000,0.000000,0.000000,1.000000,1.000000e+00
2020-04-04,0.000000,0.000000,0.000000,1.000000,1.000000e+00
2020-04-05,0.000000,0.000000,0.000000,1.000000,1.000000e+00
...,...,...,...,...,...
2023-03-22,-0.236654,0.178114,-0.058540,0.941460,-3.747275e-11
2023-03-23,0.271494,-0.178330,0.093164,1.093164,-4.096387e-11
2023-03-24,-0.178783,0.163062,-0.015720,0.984280,-4.031991e-11


In [34]:
final_ts=pd.concat([df1_long_returns['close_BTC'],final_df['return_total'],ts_short['cum_time_series_rebal_short'],ts_long['cum_time_series_rebal_long']],axis=1)
final_ts

,close_BTC,return_total,cum_time_series_rebal_short,cum_time_series_rebal_long
time,,,,
2020-03-26,NaN,NaN,NaN,NaN
2020-03-27,-0.055512,NaN,NaN,NaN
2020-03-28,-0.020416,NaN,NaN,NaN
2020-03-29,-0.059223,NaN,NaN,NaN
2020-03-30,0.088591,NaN,NaN,NaN
...,...,...,...,...
2023-03-22,-0.030817,0.941460,1.178114,0.763346
2023-03-23,0.037643,1.093164,0.821670,1.271494
2023-03-24,-0.030127,0.984280,1.163062,0.821217


In [40]:
final_ts=final_ts.replace([np.inf,-np.inf],0)
final_ts['cum_close_BTC']=1+final_ts['close_BTC']
final_ts
final=final_ts[final_ts.index>='2021-01-01']
final['cum_close_BTC'].iloc[0]=1
final['cum_time_series_rebal_long'].iloc[0]=1
final['cum_time_series_rebal_short'].iloc[0]=1
final['return_total'].iloc[0]=1

final

,close_BTC,return_total,cum_time_series_rebal_short,cum_time_series_rebal_long,cum_close_BTC
time,,,,,
2021-01-01,0.014377,1.000000,1.000000,1.000000,1.000000
2021-01-02,0.095774,1.520464,1.120429,1.400035,1.095774
2021-01-03,0.026700,0.975060,0.562849,1.412211,1.026700
2021-01-04,-0.031241,0.566602,0.733873,0.832729,0.968759
2021-01-05,0.062682,1.194841,1.004848,1.189993,1.062682
...,...,...,...,...,...
2023-03-22,-0.030817,0.941460,1.178114,0.763346,0.969183
2023-03-23,0.037643,1.093164,0.821670,1.271494,1.037643
2023-03-24,-0.030127,0.984280,1.163062,0.821217,0.969873


In [41]:
final['ts_cum_close_BTC']=final['cum_close_BTC'].cumprod()
final['ts_cum_rebal_long']=final['cum_time_series_rebal_long'].cumprod()
final['ts_cum_rebal_short']=final['cum_time_series_rebal_short'].cumprod()
final['ts_cum_return_total']=final['return_total'].cumprod()
final

,close_BTC,return_total,cum_time_series_rebal_short,cum_time_series_rebal_long,cum_close_BTC,ts_cum_close_BTC,ts_cum_rebal_long,ts_cum_rebal_short,ts_cum_return_total
time,,,,,,,,,
2021-01-01,0.014377,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00
2021-01-02,0.095774,1.520464,1.120429,1.400035,1.095774,1.095774,1.400035,1.120429e+00,1.520464e+00
2021-01-03,0.026700,0.975060,0.562849,1.412211,1.026700,1.125031,1.977146,6.306321e-01,1.482544e+00
2021-01-04,-0.031241,0.566602,0.733873,0.832729,0.968759,1.089884,1.646426,4.628038e-01,8.400116e-01
2021-01-05,0.062682,1.194841,1.004848,1.189993,1.062682,1.158201,1.959235,4.650475e-01,1.003680e+00
...,...,...,...,...,...,...,...,...,...
2023-03-22,-0.030817,0.941460,1.178114,0.763346,0.969183,0.929512,-0.004788,4.489999e-14,-2.773274e-09
2023-03-23,0.037643,1.093164,0.821670,1.271494,1.037643,0.964502,-0.006088,3.689297e-14,-3.031644e-09
2023-03-24,-0.030127,0.984280,1.163062,0.821217,0.969873,0.935445,-0.005000,4.290882e-14,-2.983986e-09


In [46]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=final.index, y=final['ts_cum_close_BTC'], name='BTC'))
fig.add_trace(go.Scatter(x=final.index, y=final['ts_cum_return_total'], name='Strat'))
# fig.add_trace(go.Scatter(x=final.index, y=final['ts_cum_rebal_short'], name='Strat_Short'))
# fig.add_trace(go.Scatter(x=final.index, y=final['ts_cum_rebal_long'], name='Strat_Long'))
fig.update_layout(
    title='Strategy Report',
    xaxis_title='Time',
    yaxis_title='Cumulative_Returns',
    hovermode='x unified',
    )

pyo.iplot(fig)